In [3]:
import tensorflow as tf
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn import metrics
from sklearn.metrics import r2_score
from tensorflow.keras import utils,losses,layers,Sequential
from tensorflow.keras.callbacks import ModelCheckpoint,TensorBoard,EarlyStopping
from tensorflow.keras.layers import Input, LSTM, Dense, Bidirectional, Dot, Softmax, Concatenate,Conv1D,Dropout,MaxPooling1D, Flatten
from tensorflow.keras.models import Model

data_path="winter.csv"
dataset=pd.read_csv(data_path,parse_dates=['Date'],index_col=['Date'])

future=['NH3']

#数据归一化，由于sin和cos本来就是-1到1，不用归一化
##Data normalization, as sin and cos are originally -1 to 1, normalization is not necessary
for col in future:
    scaler=MinMaxScaler()
    if(col not in ['sin(h)','cos(h)']):
    	dataset[col]=scaler.fit_transform(dataset[col].values.reshape(-1,1))
dataset.head()

def multivariate_data(x,y, start_index, end_index, history_size,
                      target_size, step, single_step):
    data = []
    labels = []

    start_index = start_index + history_size
    
    if end_index is None:
        end_index = len(dataset) - target_size

    for i in range(start_index, end_index):
        indices = range(i-history_size, i, step) 
        mid_data=x.iloc[indices]
        data.append(mid_data)

        if single_step:
            mid_data=y.iloc[i+target_size]
            labels.append(mid_data)
        else:
            labels.append(y.iloc[i:i+target_size])

    return np.array(data), np.array(labels)
x=dataset[future]
y=dataset['NH3']
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.3,shuffle=False,random_state=25)
train_dataset,train_labels=multivariate_data(x_train,y_train,0,2800,3,1,1,False)
test_dataset,test_labels=multivariate_data(x_test,y_test,0,1200,3,1,1,False)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.callbacks import EarlyStopping

# 构建BP神经网络模型
# Building a BP neural network model
model = Sequential([
    Flatten(input_shape=(3, 3)),  
    Dense(32, activation='relu'),
    Dense(16, activation='relu'),
    Dense(1)
])
model.compile(optimizer='adam', loss='mse')

# 设置EarlyStopping回调函数
# Set EarlyStop callback function
early_stop = EarlyStopping(monitor='val_loss', patience=5)

# 拟合模型
# Fitting model
history = model.fit(train_dataset, train_labels, epochs=128, validation_split=0.2, callbacks=[early_stop])

In [ ]:
# 使用模型进行预测
# Using models for prediction
train_predictions = model.predict(train_dataset)
test_predictions = model.predict(test_dataset)

# 反归一化
# Anti normalization
train_predictions = scaler.inverse_transform(train_predictions)
train_labels = scaler.inverse_transform(train_labels)
test_predictions = scaler.inverse_transform(test_predictions)
test_labels = scaler.inverse_transform(test_labels)
